In [ ]:
# Moluclar_Weight_Distribution_Analysis
import json
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d


def read_smiles_file(file_path):
    smiles_list = []
    with open(file_path, 'r') as file:
        for line in file:
            smiles_list.append(line.strip())
    return smiles_list

def calculate_molecular_weight(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        molecular_weight = Descriptors.MolWt(mol)
        return molecular_weight
    else:
        return None

# 计算分子量
def calculate_molecular_weights(smiles_list):
    molecular_weights = []
    for smiles in smiles_list:
        molecular_weight = calculate_molecular_weight(smiles)
        if molecular_weight is not None:
            molecular_weights.append(molecular_weight)
    return molecular_weights

# 计算分子量大于200的比例
def calculate_proportion_greater_than_200(molecular_weights):
    count_greater_than_200 = sum(1 for mw in molecular_weights if mw > 200)
    total_count = len(molecular_weights)
    return count_greater_than_200 / total_count if total_count > 0 else 0

# 绘制频率分布曲线图
def plot_molecular_weight_distribution(molecular_weights1, molecular_weights2, label1, label2, title):
    if not molecular_weights1 and not molecular_weights2:
        print("No valid molecular weights to plot.")
        return

    # 计算两组数据的频率分布
    hist1, bin_edges1 = np.histogram(molecular_weights1, bins=20, density=True)
    bin_centers1 = 0.5 * (bin_edges1[1:] + bin_edges1[:-1])

    hist2, bin_edges2 = np.histogram(molecular_weights2, bins=20, density=True)
    bin_centers2 = 0.5 * (bin_edges2[1:] + bin_edges2[:-1])

    # 生成平滑的x值
    x1 = np.linspace(min(molecular_weights1), 1500, 1000)
    x2 = np.linspace(min(molecular_weights2), 1500, 1000)

    # 插值
    interp_func1 = interp1d(bin_centers1, hist1, kind='cubic', fill_value="extrapolate")
    interp_func2 = interp1d(bin_centers2, hist2, kind='cubic', fill_value="extrapolate")
    y1 = interp_func1(x1)
    y2 = interp_func2(x2)

    # 确保频率值不小于0
    y1 = np.maximum(y1, 0)
    y2 = np.maximum(y2, 0)

    plt.figure(figsize=(10, 6))
    plt.plot(x1, y1, label=label1, color='blue')
    plt.plot(x2, y2, label=label2, color='green')
    plt.title(title)
    plt.xlabel('Molecular Weight (g/mol)')
    plt.ylabel('Probability Density')
    plt.legend()
    
    # 去掉上方和右边的框线
    ax = plt.gca()
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

    # 设置横轴范围
    plt.xlim(0, 1500)

    plt.show()

# 主函数
def main():
    file_path1 = '/home/sunhnayu/jupyterlab/XXI/img2smiles_xuexi/img2smiles/notebook/data_sample/final_unique_standard_merged.txt'  # 第一个TXT文件路径
    file_path2 = '/home/sunhnayu/jupyterlab/XXI/img2smiles_xuexi/img2smiles/notebook/data_sample/50k_random_8196_smiles.txt'  # 第二个TXT文件路径

    smiles_list1 = read_smiles_file(file_path1)
    smiles_list2 = read_smiles_file(file_path2)

    molecular_weights1 = calculate_molecular_weights(smiles_list1)
    molecular_weights2 = calculate_molecular_weights(smiles_list2)

    # 计算分子量大于200的比例
    proportion1 = calculate_proportion_greater_than_200(molecular_weights1)
    proportion2 = calculate_proportion_greater_than_200(molecular_weights2)

    print(f"Proportion of molecular weights > 200 for dataset 1: {proportion1:.2f}")
    print(f"Proportion of molecular weights > 200 for dataset 2: {proportion2:.2f}")

    # 绘制分子量分布图
    plot_molecular_weight_distribution(molecular_weights1, molecular_weights2, 'MedRDB', 'USPTO-50k', 'Distribution of Molecular Weight')

if __name__ == '__main__':
    main()